# Detroit Crime Analysis

In [1]:
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plot
import time
%matplotlib inline

## Detroit Crime Data

In [32]:
crimes = pd.read_csv('data/detroit/DPD__All_Crime_Incidents__December_6__2016_-_Present.csv', low_memory=False)
print("Length=%s" % len(crimes))

Length=132845


In [36]:
from shapely.geometry import Point

LIMIT = 500
crime_df = crimes[['uniq', 'Latitude', 'Longitude']].dropna(axis=0).head(LIMIT)
coordinates = map(Point, list(zip(crime_df.Longitude, crime_df.Latitude)))
crime_points = crime_df.assign(geometry=list(coordinates))
points = gp.GeoDataFrame(crime_points)

ax = points.plot(markersize=7)
mplleaflet.display(fig=ax.figure, crs=patrols.crs, tiles='cartodb_positron')

## Patrol Car Areas

In [3]:
patrols = gp.read_file('data/detroit/patrol-areas/geo_export_5e950023-9312-46ff-a483-bd1a77ad7c03.shp')
print("Length=%s" % len(patrols))

Length=131


In [5]:
import mplleaflet
ax = patrols.plot()
mplleaflet.display(fig=ax.figure, crs=patrols.crs, tiles='cartodb_positron')

In [28]:
centroids = patrols.geometry.centroid
ax = patrol_df.plot()
mplleaflet.display(fig=ax.figure, crs=patrols.crs, tiles='cartodb_positron')

## Calculating Distances

In [46]:
from geopy.distance import geodesic

crime_ids = list(crime_df['uniq'])
crime_xys = list(zip(crime_df.Latitude, crime_df.Longitude))

patrol_ids = list(patrols['area']) 
patrol_xys = centroids.apply(lambda c: (c.y, c.x))

In [50]:
t1 = time.time()

dists = [[geodesic(a, b) for a in patrol_xys] for b in crime_xys]

t2 = time.time()

print("Function=geodesic, Time=%s, Calls=%s" % (t2 - t1, len(patrol_xys) * len(crime_xys)))

Function=geodesic, Time=13.118151903152466, Calls=65500


In [49]:
distances = pd.DataFrame(dists, columns=patrol_ids, index=crime_ids)
distances.head()

,712,904,906,1110,1203,402,808,908,301,1007,...,804,704,709,701,708,911,401,1211,805,414
125845,7.050854557270149 km,9.080551722814157 km,11.12262320892111 km,6.237371049991145 km,11.67570838131166 km,6.1972828086609235 km,14.096735352409441 km,7.632555366283333 km,4.14796953682709 km,3.951099887939183 km,...,15.040742657839655 km,3.5021444055433997 km,3.466661579150614 km,6.019009718452777 km,5.291449009287244 km,10.804920208807898 km,7.338332707046657 km,9.371075059810591 km,18.658165723506432 km,5.016075533883745 km
125846,7.050854557270149 km,9.080551722814157 km,11.12262320892111 km,6.237371049991145 km,11.67570838131166 km,6.1972828086609235 km,14.096735352409441 km,7.632555366283333 km,4.14796953682709 km,3.951099887939183 km,...,15.040742657839655 km,3.5021444055433997 km,3.466661579150614 km,6.019009718452777 km,5.291449009287244 km,10.804920208807898 km,7.338332707046657 km,9.371075059810591 km,18.658165723506432 km,5.016075533883745 km
114360,8.575508538000133 km,5.406765685091352 km,2.4900929522468442 km,10.433855233408485 km,17.136806427100503 km,17.741824672013713 km,21.715316094044702 km,5.049447086199409 km,11.23266524431162 km,14.605479578810003 km,...,21.89350989640634 km,8.22017142476909 km,11.05105509646471 km,5.986800699927732 km,6.895158867273887 km,0.9621073502211075 km,18.884714137716628 km,16.840524458742202 km,26.90102641583579 km,15.996038571378913 km
114217,12.039444893500306 km,15.771313928212919 km,17.925050454892155 km,11.827313137686334 km,14.355572904133576 km,3.36429752667783 km,14.377191916940943 km,14.419506568359553 km,9.560814326586433 km,6.054551621831163 km,...,15.793080912115471 km,10.264954979181221 km,7.996670913110091 km,12.82160813008603 km,11.663119373118862 km,17.528302821935515 km,3.5266868352459046 km,11.313835597128847 km,17.501400979419785 km,3.073572763968789 km
114196,19.595266141890658 km,16.57031076769312 km,19.76116038807475 km,11.158777350468647 km,6.4803156883932305 km,8.632211585150852 km,2.8950653129010098 km,16.50820785403254 km,10.547955629283711 km,8.672804911306642 km,...,4.484969333485923 km,14.787851719881768 km,15.964467568962275 km,15.890570512328551 km,17.123271152195144 km,20.61428450086994 km,8.328118760849875 km,4.882556199156268 km,6.084350771477268 km,13.762200190574756 km
